In [67]:
from pathlib import Path
import os
from datetime import datetime
from datetime import timedelta
from send2trash import send2trash
import shutil

In [11]:
dir_path = '/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/'

In [17]:
current_dir = Path(dir_path)

In [31]:
most_recent = datetime.now() - timedelta(days = 30)
print(most_recent)

2021-05-30 14:48:57.090725


In [40]:
# step 1: access files
RECENT_DAYS  = 30
most_recent = datetime.now() - timedelta(days = RECENT_DAYS)
num_file = 0
num_subdir = 0
dir_size = 0
num_recent_file = 0

for entry in current_dir.iterdir():
    if entry.is_file():
        num_file += 1
        dir_size += entry.stat().st_size
        mtime = datetime.fromtimestamp(entry.stat().st_mtime)
        #print('Name %s, Time: %s' %(entry.name, mtime))
        if mtime > most_recent:
            num_recent_file += 1

    if entry.is_dir():
        num_subdir += 1
    
    
for f in current_dir.glob('**/*'):
    if f.is_file():
        dir_size += f.stat().st_size

print('Num files: %d' % num_file)
print('Recent files: %d' % num_recent_file)
print('Num subdirs: %d' % num_subdir)
print('Dir size: %d' % dir_size)

Num files: 20
Recent files: 1
Num subdirs: 3
Dir size: 25756655


In [68]:
trash_dir = current_dir.absolute().as_posix()+'/trash_bin/'

In [66]:
trash = Path(trash_dir)
trash.mkdir()

In [74]:
def create_dir(current_dir, name):
    path = current_dir.absolute().as_posix() + '/' + name
    p = Path(path)
    p.mkdir()
    return(path)

In [75]:
# step 2: manipulate files
# delete, redo, skip, move
# build a trash dir to store all the trash in the end more files to recycle bin and delete trash dir
move_dirs_path = []
move_dirs_name = []
action_list = []
trash_dir = create_dir(current_dir, 'trash_bin')

for entry in current_dir.iterdir():
    if entry.is_file():
        input_flag = True
        print('File: %s' % entry.name)
        while(input_flag):
            action = input('0: delete, 1: Pass, 2: Move 3: done')
            if action in ['0','1','2','3']:
                input_flag = False
            else:
                print('Wrong input %s  do it again' %action)
        
        if action == '0':
            # delete
            file_path = entry.absolute().as_posix()
            shutil.move(file_path,trash_dir)
            action_list.append(action)

        elif action == '1':
            # pass
            action_list.append(action)
            
        elif action == '2':
            # Move 
            file_path = entry.absolute().as_posix()
            if not move_dirs_path:
                print('There is no move dirs yet.')
                action2 = input('1: create dir 0: pass')
                if action2 == '1':
                    dir_name = input('Input dir name:')
                    move_dirs_name.append(dir_name)
                    temp_path = create_dir(current_dir, dir_name)
                    move_dirs_path.append(temp_path)
                    shutil.move(file_path,temp_path)
                    action_list.append(action)
                else:
                    action_list.append('1')
            else:
                for ii, l in enumerate(move_dirs_name):
                    print('%d  %s' %(ii, l))
                print('-1: create a new folder')
                move_dir_id = int(input('Choose which folder to move:'))
                if move_dir_id >= 0:
                    temp_path = move_dirs_path[move_dir_id]
                    shutil.move(file_path,temp_path)
                    action_list.append(action)
                else:
                    dir_name = input('Input dir name:')
                    move_dirs_name.append(dir_name)
                    temp_path = create_dir(current_dir, dir_name)
                    move_dirs_path.append(temp_path)
                    shutil.move(file_path,temp_path)
                    action_list.append(action)

        else:
            # done
            # move trash files to reycle bin and delete trash
            trash = Path(trash_dir)
            for f in trash.iterdir():
                send2trash(f.absolute().as_posix())
            trash.rmdir()

            action_list.append(action)
            break

        
        



File: uhstar_har.eps
File: err_uhstar_3d.png
File: err_uh_3d.png
There is no move dirs yet.
File: compare_num_recovered_eigenvaluesk=1vsNv2.eps
There is no move dirs yet.
File: .DS_Store
File: err_uh_2d.png
0  type1
-1: create a new folder
File: err_uhstar_2d.png
0  type1
-1: create a new folder
File: num_eig_vs_time_k=1v2.eps
File: uh_har.eps
File: ConvPlotk1Eig3.eps


In [44]:
for entry in current_dir.iterdir():
    print(entry.absolute())

/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/uhstar_har.eps
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/err_uhstar_3d.png
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/err_uh_3d.png
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/orgi_pic
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/compare_num_recovered_eigenvaluesk=1vsNv2.eps
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/.DS_Store
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/err_uh_2d.png
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/err_uhstar_2d.png
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/num_eig_vs_time_k=1v2.eps
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/uh_har.eps
/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/ConvPlotk1Eig3.eps
/Users/shiqiangxia/

In [54]:
file_path = '/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used/uhstar_har.eps'
send2trash(file_path)

In [57]:
current_dir.absolute().as_posix()

'/Users/shiqiangxia/Desktop/file_organizer_project/Experiments/pic_used'